## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

#### Данные имеют следующие столбцы

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### План работы над проектом

1. Провести анализ пропусков, изменения данных и повторения значений.
2. Определить, какие некорректные строки требуется сохранить, а какие можно удалить.
3. Заполнить значения в пропущенных ячейках таблицы
4. Удалить дубликаты.
5. Вывести категории для проведения гипотез.
6. Составить выводы по каждому вопросу.
7. Составить общий вывод.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Подключение необходимых библиотек.

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import copy as c
import numpy as np
m = Mystem()

Подключение данных и их просмотр 

In [2]:
df = pd.read_csv('/datasets/data.csv')
print(df.info())
df = df.loc[:, 'children':'purpose']
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Из полученным данных получаем следущие типы переменных по столбцам:
- Категориальные: education, education_id, family_status, family_status_id, gender, income_type, purpose.
- Количественные: children, days_employed, dob_years,total_income.
- Логические: debt.

Просмотрим пропущенные значения в таблице.

In [3]:
df[df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Проведем просмотр иникальных значений по каждой переменной.

In [4]:
print('Столбец children', df['children'].unique())
print('Столбец days_employed', df['days_employed'].unique())
print('Столбец dob_years', df['dob_years'].unique())
print('Столбец education', df['education'].unique())
print('Столбец education_id', df['education_id'].unique())
print('Столбец family_status', df['family_status'].unique())
print('Столбец family_status_id', df['family_status_id'].unique())
print('Столбец gender', df['gender'].unique())
print('Столбец income_type', df['income_type'].unique())
print('Столбец debt', df['debt'].unique())
print('Столбец total_income', df['total_income'].unique())
print('Столбец purpose', df['purpose'].unique())

Столбец children [ 1  0  3  2 -1  4 20  5]
Столбец days_employed [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
Столбец dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
Столбец education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Столбец education_id [0 1 2 3 4]
Столбец family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Столбец family_status_id [0 1 2 3 4]
Столбец gender ['F' 'M' 'XNA']
Столбец income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Столбец debt [0 1]
Столбец total_income [253875.6394526  112080.014

In [5]:
df[df['gender'] == 'XNA'].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


### Вывод

Данные имеют аномалии в виде:
1. Столбец Unnamed:0. Повтор индексов DataFrame. Появился один раз при загрузке данных из таблицы. Но в любом случае берутся только информативные столбцы. Случайный тип ошибки.
2. 20 и -1 детей. Первое могло быть связано с печатной ошибкой или переводом из рукописного ввода в печатный. Второе говорит об отсутвии детей. Так как самих значений мало, скорей всего это носит случайный характер.
3. Отрицательное количество дней работы. Это может быть связано с расчетом количества дней работы с момента первой работы по текущий или последний день работы. Это неслучайный характер ошибки.
4. Положительное количество дней работы. Может быть связано с тем, что вычисленно количество часов с момента первой работы по текущий момент. Это неслучайный характер ошибки.
5. Наличие дубликатов. В системе не созданы оптимальные критерии по названию как результат есть три способа написания Среднего общего образования. Это неслучайный характер ошибки, т.к. каждый раз оператор вводил агента образования по разному.
6. Есть пропуски возраста. Клиент не передал информацию о поле возраст. Это неслучайный характер ошибки.
7. Пропущены строки с днями работы и текущей зарплатой. Клиенты не передали информацию в этих позициях. Это может нести случайный характер, т.к. невозможно выдать кредит, не имея представления о заработной плате. Пропуски в з/п и днях находяться у одних и тех же пользователей.
9. Значение XNA в столбце gender. Клиент не передал информацию о поле при выборе M и F. Это случайный характер ошибки.

### Шаг 2. Предобработка данных

### Обработка пропусков

#### Пропуск в столбце gender. 

Было выставлено значение 'M' соответсвующее мужскому полу по следующим причинам.

- Возраст и неоконченное высшее предполагает, что человек работает паралельно с учебой. Стаж работы предполагает, что человек начал свою карьеру в 17-18 лет.
- Наличие гражданского брака и статус в компании говорит о его дружеских связях, или что он один из первых соучередителей.

In [6]:
df.loc[10701, 'gender'] = 'M'

df.loc[10701, 'gender']

'M'

#### Пропуски в столбце days_employed и total_income	.

Перед проведением анализа необходимо изменить данные в столбце days_employed:

- Положительные значения поделить на 24, количество часов в сутках
- Отрицательные перевести в положительные

In [7]:
def converse_days_employed(row):
    if row> 0:
        row = row / 24
        return row
    elif row < 0:
        row = row * -1
        return row
    else:
        return row

df1 = c.deepcopy(df)
print (df1['days_employed'].head())

df1.dropna(inplace=True)

df1['days_employed'] = df1['days_employed'].apply(converse_days_employed)
print (df1['days_employed'].head())

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
Name: days_employed, dtype: float64
0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4    14177.753002
Name: days_employed, dtype: float64


Выявим закономерность между типом занятости и количеством дней работы с ежемесячным доходом на примере типа занятости "сотрудник".

In [8]:
worker_days_employed = df1['days_employed'][df1['income_type'] == 'сотрудник']
worker_days_employed.head()

0    8437.673028
1    4024.803754
2    5623.422610
3    4124.747207
7     152.779569
Name: days_employed, dtype: float64

In [9]:
worker_days_employed_mean = worker_days_employed.mean()
worker_days_employed_median = worker_days_employed.median()
print(worker_days_employed_mean, worker_days_employed_median, worker_days_employed_mean/worker_days_employed_median)

2326.4992159717935 1574.2028211070851 1.4778903866629098


In [10]:
worker_total_income = df1['total_income'][df1['income_type'] == 'сотрудник']
worker_total_income.head()

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
7    135823.934197
Name: total_income, dtype: float64

In [11]:
worker_total_income_mean = worker_total_income.mean()
worker_total_income_median = worker_total_income.median()
print(worker_total_income_mean, worker_total_income_median, worker_total_income_mean/worker_total_income_median)

161380.2604878855 142594.39684740017 1.1317433507614563


Средние значения по выборке завышены за счет больших значений в ней. Будем ориентироваться на медиану для случая количества рабочих дней. И использовать среднее значение для дохода в месяц, если среднее не больше медианы на 20%. Ведь чем сильнее среднее значение отличается от медианы, тем больше разница верхних и нижних значений.

In [12]:
#print(df.loc[82,:])

#df['days_employed'] = df['days_employed'].fillna(worker_days_employed_median)
#df['total_income'] = df['total_income'].fillna(worker_total_income_median)

#print(df.loc[82,:])

Проведем заполнение пустот для остальных типов дохода. Будем использовать среднее значение если оно не превышает на 20% медиану. Найдем все значения для каждой категории дохода, которые имеют пропуски.

In [13]:
df[df['days_employed'].isna()]['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

In [14]:
def mean_or_median(df, column, income_type):
    data = df[column][df['income_type'] == income_type]
    data = df[column]
    data_mean = data.mean()
    data_median = data.median()
    if data_mean / data_median > 1.2:
        return data_median
    else :
        return data_mean
    
#'сотрудник' - worker 
#'пенсионер' - pensioner
#'компаньон' - partner
#'госслужащий' - clerk
#'предприниматель' - businessman  

worker_days_fillna_value = mean_or_median(df1, 'days_employed', 'сотрудник')
worker_income_fillna_value = mean_or_median(df1, 'total_income', 'сотрудник')

pensioner_days_fillna_value = mean_or_median(df1, 'days_employed', 'пенсионер')
pensioner_income_fillna_value = mean_or_median(df1, 'total_income', 'пенсионер')

partner_days_fillna_value = mean_or_median(df1, 'days_employed', 'компаньон')
partner_income_fillna_value = mean_or_median(df1, 'total_income', 'компаньон')

clerk_days_fillna_value = mean_or_median(df1, 'days_employed', 'госслужащий')
clerk_income_fillna_value = mean_or_median(df1, 'total_income', 'госслужащий')

businessman_days_fillna_value = mean_or_median(df1, 'days_employed', 'предприниматель')
businessman_income_fillna_value = mean_or_median(df1, 'total_income', 'предприниматель')

fill_dict = {'сотрудник': [worker_days_fillna_value, worker_income_fillna_value],
             'пенсионер': [pensioner_days_fillna_value, pensioner_income_fillna_value],
             'компаньон': [partner_days_fillna_value, partner_income_fillna_value],
             'госслужащий': [clerk_days_fillna_value, clerk_income_fillna_value],
             'предприниматель': [businessman_days_fillna_value, businessman_income_fillna_value]}

In [15]:
print(df.info())
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0)

def fill_days_and_income(row, fill_dict):
    if row[1] == 0:
        row[1] = fill_dict[row[8]][0]
        row[10] = fill_dict[row[8]][1]
    return row

df = df.apply(fill_days_and_income, axis=1, args=[fill_dict])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family

In [16]:
df[df['days_employed'].isna()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Нулевое значение в столбце dob_years.

Восстановление можно сделать по знанию количества дней работы человека. К сожалению, в выборке количество дней работы не всегда соотвествуют возрасту старта работы в 18 лет. На указанные гипотезы возраст не влияет, поэтому заполним пустоты средним значение по выборке. Тип int был применен, чтобы не изменить хранимый тип данных в столбце.

In [17]:
df['dob_years'] = df['dob_years'].replace(0, int(df['dob_years'][df['dob_years'] != 0].mean()))

df['dob_years'][df['dob_years'] == 0].count()

0

#### Значения 20 и -1 в столбце children.

20 заменим на 2, это ошибка перевода в датасет. Человек подразумевал 2 ребенка.
-1 заменим на 0, т.к. скорей подразумевалось отсутсвие детей.

In [18]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 0)

### Вывод

Данные обладали аномалиями, которые были решены:

- Пропуск в столбце gender. 
- Отрицательные и большие положительные значения в столбце days_employed.
- Пропуски в столбцах days_employed и total_income.
- Нулевое значение в столбце dob_years.

Пояснения по выбору были приведены по ходу составления кода.

### Замена типа данных

Замены требуют столбцы days_employed и total_income. Дробная часть в days_employed появляется из наличия часов работы; в total_income из-за налогового вычета. Для гипотез дробная часть несущественна.

In [19]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Данные извенены с float на int из-за несущественности дробной часть по сравнению с целой.

### Обработка дубликатов

Проведем просмотр уникальных значений по столбкам для каждой категориальной переменной.

In [20]:
print(df['education'].unique())
df.groupby('education')['education'].count()

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


education
ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

Приведем все названия в столбцах к нижнему регистру.

In [21]:
df['education'] = df['education'].str.lower()
print(df['education'].unique())
df.groupby('education')['education'].count()

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

Проверим уникальные значени и их количество по качественным критериям.

In [22]:
print(df['children'].value_counts())
print(df['dob_years'].value_counts())
print(df['education'].value_counts())
print(df['education_id'].value_counts())
print(df['family_status'].value_counts())
print(df['family_status_id'].value_counts())
print(df['gender'].value_counts())
print(df['income_type'].value_counts())
print(df['debt'].value_counts())
print(df['purpose'].value_counts())

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64
35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64
женат / замужем          12380
гражданский

In [23]:
print(df.info())
df = df.drop_duplicates()
print(df.info())
df.tail(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       2145

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21505,0,338904,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439,сыграть свадьбу
21506,1,-1556,33,высшее,0,гражданский брак,1,F,сотрудник,0,145541,свадьба
21507,1,-79,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180,сделка с подержанным автомобилем
21508,0,386497,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638,недвижимость
21509,0,362161,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029,операции с недвижимостью
21510,2,2194,28,среднее,1,женат / замужем,0,F,сотрудник,0,167422,приобретение автомобиля
21511,0,-612,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068,покупка жилья для сдачи
21512,0,-165,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301,получение дополнительного образования
21513,0,-1166,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986,покупка жилья
21514,0,-280,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988,строительство недвижимости


### Вывод

Дубликаты исчезли из столца. Теперь количество уникальных типов образований соотвествует количеству значени из столца education_id.


### Лемматизация

Столбец purpose требует выявления общих категорий для одного типа целей кредита.

In [24]:
unique_purpuse = df['purpose'].unique() 
purpuse_set = []
for i in unique_purpuse:
    print(m.lemmatize(i))
    #if 'ремонт' in m.lemmatize(i):
        #print(m.lemmatize(i))
    for j in m.lemmatize(i):
        purpuse_set.append(j)
purpuse_set = set(purpuse_set)
purpuse_set

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

{'\n',
 ' ',
 'автомобиль',
 'высокий',
 'для',
 'дополнительный',
 'жилой',
 'жилье',
 'заниматься',
 'коммерческий',
 'на',
 'недвижимость',
 'образование',
 'операция',
 'подержанный',
 'подержать',
 'покупка',
 'получение',
 'приобретение',
 'проведение',
 'профильный',
 'ремонт',
 'с',
 'свадьба',
 'свой',
 'сдача',
 'сделка',
 'семья',
 'со',
 'собственный',
 'строительство',
 'сыграть'}

Из данных по лемматизиции можно получить следующие пять категорий:

- 'свадьба' - все, что относится к проведению или организацию свадьбы.
- 'автомобиль' - покупка или ремонт машины.
- 'образование' - приобретение или оплата обучения.
- 'жилая недвижимость' - действия связанные с покупкой или ремонтом жилья.
- 'коммерческая недвижимость' - действия связанные с недвижимостью, сданную под бизнесс. Если не был описан тип недвижимости, он будет считаться коммерческим.

На основе данной информации создадим столбец purpose_id, в котором распределим значения этих категорий:

- 'свадьба' = 0
- 'автомобиль' = 1
- 'образование' = 2
- 'жилая недвижимость' = 3
- 'коммерческая недвижимость' = 4

Определим словарь для для категорий.

---
## Комментарий от наставника

Все верно, метод lemmatize() использован правильно. Хорошо бы подробнее объяснить выбор лемм.

---

In [25]:
purpose_dict = {'свадьба':0, 'автомобиль':1, 'образование':2, 'жилой':3, 'жилье':3, 'собственный':3,  'коммерческий':4, 'недвижимость':4}

In [26]:
print(df.info())

def set_purpose_id(row, purpose_dict):
    lemma = m.lemmatize(row)
    for word in lemma:
        if word in purpose_dict:
            return purpose_dict[word]
        

df['purpose_id'] = df['purpose'].apply(set_purpose_id, args=[purpose_dict])
        
print(df.info())

print(df[df['purpose_id'].isna()])

df['purpose_id'].value_counts() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       2145

3    6325
4    4486
1    4306
2    4013
0    2324
Name: purpose_id, dtype: int64

### Вывод

Лемматизация позволила упростить категоризацию цели кредита для оценки гипотез. Больше всего кредитов просят на операции с недвижимостью.

### Категоризация данных

Категории позволяют упростить подтвержение или опровержение гипотезы. Для гипотез уже готовы категории:

- Семейное положение
- Цель кредита
- Количество детей в семье

Остальные категории, которые нужно создать и задать разделение:

- Уровень дохода: низкий, средний, высокий.

Начнем с категоризации семей.

In [27]:
print(df['children'].value_counts())

def category_children(row):
    if row == 0:
        return 'нет детей'
    elif row < 3:
        return 'малая семья'
    else:
        return 'многодетная семья'
    
df['category_children'] = df['children'].apply(category_children)
df['category_children'].value_counts()


0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


нет детей            14138
малая семья           6936
многодетная семья      380
Name: category_children, dtype: int64

Категоризация уровня дохода.

Одним из способов категоризации доходов является соотнесение с медианным доходом. Проведем расчеты с вычислением медианы по доходу в месяц и построение распределение уровней дохода следущим образом:

- Нижний средний класс до 0,75 медианы
- Средний класс от 0,75 до 1,25 медианы
- Верхний средний класс от 1,25 до 2 медиан
- Обеспеченный класс от 2 медиан

Вычислим медиану по выборке

In [28]:
df['total_income'].median()

156043.5

Возьмем за 1 медиану зарплату в 160 тысяч. Исходя из этого получаем пределы:
    
- Доход до 120 тысяч
- Доход от 120 до 200 тысяч
- Доход от 200 до 320 тысяч
- Доход от 320 тысяч

In [29]:
print(df['total_income'].sort_values())

def category_total_income(row):
    if row < 120000:
        return 'Доход до 120 тысяч'
    elif row < 200000:
        return 'Доход от 120 до 200 тысяч'
    elif row < 320000:
        return 'Доход от 200 до 320 тысяч'   
    else:
        return 'Доход от 320 тысяч'
    
df['category_total_income'] = df['total_income'].apply(category_total_income)
df['category_total_income'].value_counts()

14585      20667
13006      21205
16174      21367
1598       21695
14276      21895
          ...   
17178    1711309
20809    1715018
9169     1726276
19606    2200852
12412    2265604
Name: total_income, Length: 21454, dtype: int64


Доход от 120 до 200 тысяч    9543
Доход до 120 тысяч           6845
Доход от 200 до 320 тысяч    3844
Доход от 320 тысяч           1222
Name: category_total_income, dtype: int64

Добавим словестную категорию для целей кредита через словарь и этот столбец занесем в таблицу.

In [30]:
purpose_dict = {0: 'свадьба', 1: 'автомобиль', 2: 'образование', 3: 'жилая недвижимость', 4: 'коммерческая недвижимость'}

def category_purpose(row):
    return purpose_dict[row]

df['category_purpose'] = df['purpose_id'].apply(category_purpose)
df['category_purpose'].value_counts()

жилая недвижимость           6325
коммерческая недвижимость    4486
автомобиль                   4306
образование                  4013
свадьба                      2324
Name: category_purpose, dtype: int64

### Вывод

Данные подготовлены к проверке гипотез.

- Категория по семейному положению имелась в начале.
- Категория по цели кредита создалась по лемматизации запросов. Причины объясненены выше.
- Категория по семье создана по следующему принципу: отсутвие детей не накладывают дополниельных ограничений. Один или два ребенка - нормальные ограничения для семьи. Многодетные семьи в большей мере живут для поддержания своего уровня жизни и накладывают сильные ограничения.
- Категория по уровню дохода в месяц. 

### Шаг 3. Ответьте на вопросы

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
table_1 = pd.pivot_table(df, index='category_children', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='category_children', values='debt', aggfunc='sum')
dept_by_childrens = table_1
dept_by_childrens['non_returned_debt'] = table_2['debt']
dept_by_childrens['returned_debt'] = dept_by_childrens['debt'] - dept_by_childrens['non_returned_debt']
dept_by_childrens['return_ratio'] = dept_by_childrens['returned_debt'] / dept_by_childrens['debt']
dept_by_childrens

,debt,non_returned_debt,returned_debt,return_ratio
category_children,,,,
малая семья,6936,646,6290,0.906863
многодетная семья,380,31,349,0.918421
нет детей,14138,1064,13074,0.924742


### Вывод

Имеется зависимость между количество детей и возвратом кредита в срок.

1. Самое высокое значение имеют семьи без детей - 92,47% возврата
2. За ними идут многодетные семьи - 91,84% возврата
3. Малые семьи из 1 или 2 детей  - 90,69% возврата

Эту зависимость можно пояснить следующим образом:

- Семьи без детей не имет случайных издержек при содержании детей и все их деньги приходяться только на одного или двух человек. Соответсвенно, контролировать возврат кредита проще.
- Многодетные семьи принимают всю ответсвенность за свою жизнь и своих детей и стараются меньше брать кредитов и больше их возвращать в срок, чтобы у низ была возмодность расти дальше без долга.
- Малые семьи еще не имеют опыта в распределении финансов на 3-4 человека в семье и иногда не рассчитывают свои возможности из-за случаных и регулярных издержек содержания ребенка.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
table_1 = pd.pivot_table(df, index='family_status', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='family_status', values='debt', aggfunc='sum')
dept_by_family_status = table_1
dept_by_family_status['non_returned_debt'] = table_2['debt']
dept_by_family_status['returned_debt'] = dept_by_family_status['debt'] - dept_by_family_status['non_returned_debt']
dept_by_family_status['return_ratio'] = dept_by_family_status['returned_debt'] / dept_by_family_status['debt']
dept_by_family_status

,debt,non_returned_debt,returned_debt,return_ratio
family_status,,,,
Не женат / не замужем,2810,274,2536,0.902491
в разводе,1195,85,1110,0.928870
вдовец / вдова,959,63,896,0.934307
гражданский брак,4151,388,3763,0.906529
женат / замужем,12339,931,11408,0.924548


### Вывод

Имеется зависимость между семейным положением и возвратом кредита в срок.

1. Самый высокий процент показали "вдовец/вдова" - 93,43%
2. Следом категория "в разводе" - 92,88%
3. Далее "женат / замужем" - 92,45%
4. "Гражданский брак" - 90,65%
5. И "Не женат / не замужем" - 90,24%

Градацию можно объяснить следующими факторами:

- Люди, которые остались одни в жизни или развелись теперь несут ответсвенность в одиночку и отдают себе отчет за контроль денег, т.к. расчитывать можно только на себя.
- Те, кто живут и собираются жить дальше вместе, понимают уровень отвественности при взятии кредита, т.к. им необходимо готовиться к расширению семьи или уже имеют достаточно детей.
- Остальные же пока не осознают полную ответсвенность за себя и свою жизнь. И за деньги в том числе.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
table_1 = pd.pivot_table(df, index='category_total_income', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='category_total_income', values='debt', aggfunc='sum')
dept_by_total_income = table_1
dept_by_total_income['non_returned_debt'] = table_2['debt']
dept_by_total_income['returned_debt'] = dept_by_total_income['debt'] - dept_by_total_income['non_returned_debt']
dept_by_total_income['return_ratio'] = dept_by_total_income['returned_debt'] / dept_by_total_income['debt']
dept_by_total_income

,debt,non_returned_debt,returned_debt,return_ratio
category_total_income,,,,
Доход до 120 тысяч,6845,551,6294,0.919503
Доход от 120 до 200 тысяч,9543,832,8711,0.912816
Доход от 200 до 320 тысяч,3844,273,3571,0.928980
Доход от 320 тысяч,1222,85,1137,0.930442


### Вывод

Имеется зависимость между уровнем дохода и возвратом кредита в срок.

1. На первом месте уровень дохода от 320 тысяч - 93,04%
2. На втором месте доход от 200 до 320 тысяч - 92,9%
3. Следом доход до 120 тысяч - 91,95%
4. Потом доход от 120 до 200 тысяч - 91,28%


Все это объясняется следующим образом:

- Люди с с доходом от 200 тысяч имеют высокий уровень воврата из-за больших возможностей по поддержанию своего уровня жизни. Случаи невозврата скорей всего связаны с выплатой нескольких кредитов. Для категории от 200 до 320 тысяч возможени боолее низкий процент из-за перехода на новый стиль жизни, в котором расходы больше прежнего.
- Доход до 120 тысяч обладает большим процентом над доходом от 120 до 200 тысяч по той причине, что люди с меньшим доходам стараются планировать расходы, чтобы не быть в долгу слишком часто.


#### Как разные цели кредита влияют на его возврат в срок?

In [34]:
table_1 = pd.pivot_table(df, index='category_purpose', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='category_purpose', values='debt', aggfunc='sum')
dept_by_purpose = table_1
dept_by_purpose['non_returned_debt'] = table_2['debt']
dept_by_purpose['returned_debt'] = dept_by_purpose['debt'] - dept_by_purpose['non_returned_debt']
dept_by_purpose['return_ratio'] = dept_by_purpose['returned_debt'] / dept_by_purpose['debt']
dept_by_purpose

,debt,non_returned_debt,returned_debt,return_ratio
category_purpose,,,,
автомобиль,4306,403,3903,0.906410
жилая недвижимость,6325,439,5886,0.930593
коммерческая недвижимость,4486,343,4143,0.923540
образование,4013,370,3643,0.907800
свадьба,2324,186,2138,0.919966


### Вывод

Имеется зависимость между целью и возвратом кредита в срок.

1. Самый популярный тип кредита и самый возвращаемый - жилая недвижимость - 93,06%
2. Затем коммерческая недвижимость - 92,35%
3. За ней свадьба - 92%
4. Образование - 90,78%
5. И автомобиль	- 90.64%

Причины распределения:

- Строительство, покупка или ремонт жилья - это уровень качества жизни людей. К этому вопросу они относятся ответсвеннней всего и не хотят долго находиться в долговой зависимости.
- Комерческая недвижимость - один из способов дохода. Этим можно пояснить столь высокий процент возврата. Иногда случаются издержки, которые мешают вовремя сделать выплату по кредиту.
- Свадьба обычно дорогое событие, но с помощью родственников и друзей удается эффективно возвращать долг.
- Образование позволяет людям расширять своим возможности. К сожалению, оно также забирает у них время, чтобы иметь достаточный доход для возврата кредита в срок.
- Автомобиль получил свою позицию оправданно. Это источник дополнительных расходов: бензин, техническое обслуживание, страховка и возможные ДТП. Как результат, обычно самый сложная к возврату цель, т.к. дохоода может не хватать для жизни и для кредита.

### Шаг 4. Общий вывод

Перед окончательным выводом добавим рейтинг возврата по образованию и возрасту.

In [35]:
table_1 = pd.pivot_table(df, index='education', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='education', values='debt', aggfunc='sum')
dept_by_education = table_1
dept_by_education['non_returned_debt'] = table_2['debt']
dept_by_education['returned_debt'] = dept_by_education['debt'] - dept_by_education['non_returned_debt']
dept_by_education['return_ratio'] = dept_by_education['returned_debt'] / dept_by_education['debt']
dept_by_education

,debt,non_returned_debt,returned_debt,return_ratio
education,,,,
высшее,5250,278,4972,0.947048
начальное,282,31,251,0.890071
неоконченное высшее,744,68,676,0.908602
среднее,15172,1364,13808,0.910098
ученая степень,6,0,6,1.000000


In [36]:
#print(df['dob_years'].sort_values())

def dob_years(row):
    if row < 30:
        return 'до 30 лет'
    elif row < 40:
        return '30-39 лет'
    elif row < 50:
        return '40-49 лет'
    elif row < 60:
        return '50-59 лет'
    else:
        return 'от 60 лет'
    
df['category_dob_years'] = df['dob_years'].apply(dob_years)
df['category_dob_years'].value_counts()

table_1 = pd.pivot_table(df, index='category_dob_years', values='debt', aggfunc='count')
table_2 = pd.pivot_table(df, index='category_dob_years', values='debt', aggfunc='sum')
dept_by_dob_years = table_1
dept_by_dob_years['non_returned_debt'] = table_2['debt']
dept_by_dob_years['returned_debt'] = dept_by_dob_years['debt'] - dept_by_dob_years['non_returned_debt']
dept_by_dob_years['return_ratio'] = dept_by_dob_years['returned_debt'] / dept_by_dob_years['debt']
dept_by_dob_years

,debt,non_returned_debt,returned_debt,return_ratio
category_dob_years,,,,
30-39 лет,5662,552,5110,0.902508
40-49 лет,5455,412,5043,0.924473
50-59 лет,4657,305,4352,0.934507
до 30 лет,3180,349,2831,0.890252
от 60 лет,2500,123,2377,0.950800


Проведенная работа над исследованием надёжности заёмщиков завершена.

На основе имеющихся таблиц вывести следущие закономерности:

- Чем выше и лучше твое образование, тем больше ответсвенности и понимания приходит по поводу кредита. 
- С возрастом приходит понимание как распределять деньги и возвращать кредит вовремя.
- Наличие детей в целом негативно влияет на возврат кредита.
- Если люди живут только ради себя, вероятность невозврата кредита увеличивается.
- Люди с высоким доходом успешнее возвращают кредит, почти так же, как и люди с низким доходом.
- Цель кредита, повышающий качество жизни без серьезных расходов - повышает возврат кредита.

Опишем идеального кандидата на возврат кредита по данным этой выборки без учета возраста и образования:
Человек, потерявщий супруга, не имеет детей, с доходом от 320 тысяч, который делает операции с жилой недвижимостью. Человек, обладающий опытом жизни и понимающий, что он может расчитывать только на себя и при его скромных доходах он эффективно планирует возврат кредита.

Кандидат с наименьшой вероятностью по выдачи кредита:
Человек, живущий один, с доходом от 120 до 200 тысяч желает купить машину в кредит.

Проект состоял из следущих задач, которые успешно выполнены:

1. Оценка аномалий и определение их типа.
2. Выявление пропусков и их заполнение, замена некорректных значений.
3. Приведение дублирующих названий в один вид и удаление полных дупликатов.
4. Лемматизация целей дохода и выявление категорий.
5. Проведение сводного анализа для подтверждение гипотез.
6. Получение опыта в распределении доходов по миру